# アナリストレーティングとリビジョンでマトリックスをつくってパフォーマンス分析

[日本株のバブル相場、プロがひっそり儲けている「凄すぎる投資法」の正体（大川 智宏） | マネー現代 | 講談社（4/6）](https://gendai.ismedia.jp/articles/-/73281?page=4)

![](https://gendai.ismcdn.jp/mwimgs/8/0/-/img_80795e31824430178f25792d311a24c757495.jpg)

記事によると


![](https://gendai.ismcdn.jp/mwimgs/e/0/-/img_e028d1d1cbff33aba8922596fb18d62859851.jpg)

---

[Quantopian Docs: FactSet Estimates - Consensus](https://www.quantopian.com/docs/data-reference/estimates_consensus#factset-estimates-consensus-data-reference)
+ [Fields](https://www.quantopian.com/docs/data-reference/estimates_consensus#fields)
    + `up` (dtype float64) - The number of estimates revised up within FactSet’s consensus window for the specified slice.
    + `down` (dtype float64) - The number of estimates revised down within FactSet’s consensus window for the specified slice.
    + `num_est` (dtype float64) - The number of estimates included in the consensus values, forecasted within FactSet’s consensus window for the specified slice.

[Quantopian Docs: FactSet Estimates - Broker Recommendations](https://www.quantopian.com/docs/data-reference/estimates_recommendations)

Rating|Description
---|---
1|Buy
1.5|Overweight
2|Hold
2.5|Underweight
3|Sell


In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.factset.estimates import PeriodicConsensus
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.research import run_pipeline

fq1_eps_cons = PeriodicConsensus.slice('EPS', 'qf', 1)
fq2_eps_cons = PeriodicConsensus.slice('EPS', 'qf', 2)

fq1_eps_mean = fq1_eps_cons.mean.latest
fq2_eps_mean = fq2_eps_cons.mean.latest

estimated_growth_factor = (fq2_eps_mean - fq1_eps_mean) / fq1_eps_mean

pipe = Pipeline(
    columns={
        'fq1_eps_mean':fq1_eps_mean,
        'fq2_eps_mean':fq2_eps_mean,
        'estimated_growth_factor': estimated_growth_factor,
    },
    domain=US_EQUITIES,
    screen=estimated_growth_factor.notnull(),
)


df = run_pipeline(pipe, '2018-05-05', '2019-05-05')


In [ ]:
from quantopian.pipeline import Pipeline
import quantopian.pipeline.data.factset.estimates as fe
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.research import run_pipeline

# Create a reference to the ConsensusRecommendations dataset.
fe_rec = fe.ConsensusRecommendations

# Create a pipeline that gets the most recent recommendations for
# all US equities.
pipe = Pipeline(
    columns={
        'rec_buy': fe_rec.buy.latest,
        'rec_overweight': fe_rec.over.latest,
        'rec_hold': fe_rec.hold.latest,
        'rec_underweight': fe_rec.under.latest,
        'rec_sell': fe_rec.sell.latest,
        'rec_total': fe_rec.total.latest,
        'no_rec': fe_rec.no_rec.latest,
        'rec_mark': fe_rec.mark.latest,
        'rec_asof': fe_rec.asof_date.latest,
        'rec_ts': fe_rec.timestamp.latest,
    },
    domain=US_EQUITIES,
    screen=fe_rec.total.latest.notnull(),
)

# Run the pipeline over a year and print the first few rows of the result.
df = run_pipeline(pipe,'2004-05-05', '2019-05-05')

In [ ]:
df.head()